Ce script fusionne les associations Métiers de l'Onisep - Code ROME faite par Impala et par l'Onisep.

In [2]:
import os
import json
import numpy as np
import math
import pandas as pd

In [38]:
# Chemins utiles
dir_path = '/home/timothee/PycharmProjects/ImpalaPoleEmploi2/Impala/Onisep tSNE et clustering'
# Dossier data
data_dir_path = os.path.join(dir_path, '0 - data')
# Fichier Onisep_vs_PE.csv
onisep_vs_pe_path = os.path.join(data_dir_path, 'ONISEP_vs_PE.csv')
# Dossier Scraping 
scraping_dir_path = os.path.join(dir_path, '00 - Scraping output')
# Dossier Parsing
parsing_dir_path = os.path.join(dir_path, '00 - Parsing output')
# Liste des métiers
scraping_list = os.listdir(scraping_dir_path)
parsing_list = os.listdir(parsing_dir_path)

In [5]:
parsing_path = os.path.join(parsing_dir_path, parsing_list[1])
with open(parsing_path) as f:
    parse = json.load(f)
    f.close

list(parse.keys())

["Secteurs d'activité",
 'Vie professionnelle',
 'Métier',
 'rome',
 'Accès au métier',
 'Fonctions',
 'Statuts',
 'Descriptif',
 'lien_onisep',
 "Centres d'intérêt",
 'Groupe Formation Emploi',
 'Disciplines',
 'Métiers associés']

In [7]:
# Chargement des liens
onisep_link_path = os.path.join(data_dir_path, 'Onisep_metier_link.csv')
df = pd.read_csv(onisep_link_path, sep =";", header=0)
df.columns=["metier_onisep","lien_onisep"]

code_onisep = []
for row in df.itertuples():
    link = row[2]
    identifiant = link[-5:-1] + link[-1]
    if identifiant[0]!='1':
        identifiant = '1' + identifiant
    code_onisep.append(identifiant)
    
df["code_onisep"] = pd.Series(code_onisep)
df.head()

,metier_onisep,lien_onisep,code_onisep
0,technicien/ne de fabrication de mobilier et de...,http://www.onisep.fr/http/redirection/metier/i...,10002
1,agenceur/euse de cuisines et salles de bains,http://www.onisep.fr/http/redirection/metier/i...,10003
2,chef/fe de fabrication des industries graphiques,http://www.onisep.fr/http/redirection/metier/i...,10005
3,conducteur/trice de machines à imprimer,http://www.onisep.fr/http/redirection/metier/i...,10007
4,opérateur/trice prépresse,http://www.onisep.fr/http/redirection/metier/i...,10008


In [35]:
n = len(scraping_list)
for i in range(n):
    # Ajout de l'intitulé du métier du site de l'Onisep
    identifiant_scraping = scraping_list[i][0:6]
    if identifiant_scraping[5]=='.':
        identifiant_scraping = identifiant_scraping[0:5]
    index_scraping = df[df["code_onisep"]==identifiant_scraping].index
    
    scraping_path = os.path.join(scraping_dir_path, scraping_list[i])
    with open(scraping_path) as f:
        scrap = json.load(f)
        f.close

    intitule_site_onisep = scrap["Métier"]
    df.loc[index_scraping,"intitule_site_onisep"] = intitule_site_onisep
    
    
for i in range(n+2): 
    # Ajout de l'intitulé du métier des fiches XML envoyés par l'Onisep 
    # et ajout de l'association code ROME - fiche métier faite par l'Onisep
    identifiant = parsing_list[i][0:6]
    if identifiant[5]=='.':
        identifiant = identifiant[0:5]
    index = df[df["code_onisep"]==identifiant].index
    parsing_path = os.path.join(parsing_dir_path, parsing_list[i])
    with open(parsing_path) as f:
        parse = json.load(f)
        f.close
    
    intitule_XML = parse["Métier"]
    df.loc[index,"intitule_XML"] = intitule_XML
    
    rome = parse["rome"]
    j = 0
    for code_rome, intitule_rome in rome.items():
        j += 1
        df.loc[index,"code_ROME_" + str(j)] = code_rome
        df.loc[index,"intitule_ROME_"+ str(j)] = intitule_rome

In [40]:
# Association code ROME - métier de l'Onisep faite par Antoine et Adel
onisep_pe = pd.read_csv(onisep_vs_pe_path)
onisep_pe.head()

,Metier,code ROME
0,accessoiriste,L1503
1,accompagnateur / accompagnatrice de voyages,G1201
2,acheteur / acheteuse,M1101
3,acheteur / acheteuse d'espaces publicitaires,E1402
4,actuaire,C1105


In [41]:
# Ajout dans le data frame
for row in onisep_pe.itertuples():
    metier = row[1]
    code_rome = row[2]
    index = df[df["intitule_site_onisep"]==metier].index
    df.loc[index,"code_ROME (Impala)"] = code_rome

In [42]:
# Aperçu final du data frame
df.head()

,metier_onisep,lien_onisep,code_onisep,intitule_site_onisep,intitule_XML,code_ROME_1,intitule_ROME_1,code_ROME_2,intitule_ROME_2,code_ROME_3,intitule_ROME_3,code_ROME_4,intitule_ROME_4,code_ROME_5,intitule_ROME_5,code_ROME (Impala)
0,technicien/ne de fabrication de mobilier et de...,http://www.onisep.fr/http/redirection/metier/i...,10002,technicien / technicienne de fabrication de mo...,technicien/ne de fabrication de mobilier et de...,H2209,Intervention technique en ameublement et bois,H2204,Encadrement des industries de l'ameublement et...,NaN,NaN,NaN,NaN,NaN,NaN,H2202
1,agenceur/euse de cuisines et salles de bains,http://www.onisep.fr/http/redirection/metier/i...,10003,agenceur / agenceuse de cuisines et salles de ...,agenceur/euse de cuisines et salles de bains,F1604,Montage d'agencements,H2206,Réalisation de menuiserie bois et tonnellerie,NaN,NaN,NaN,NaN,NaN,NaN,H2206
2,chef/fe de fabrication des industries graphiques,http://www.onisep.fr/http/redirection/metier/i...,10005,chef / cheffe de fabrication des industries gr...,chef/fe de fabrication des industries graphiques,E1303,Encadrement des industries graphiques,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,conducteur/trice de machines à imprimer,http://www.onisep.fr/http/redirection/metier/i...,10007,conducteur / conductrice de machines à imprimer,conducteur/trice de machines à imprimer,E1301,Conduite de machines d'impression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,opérateur/trice prépresse,http://www.onisep.fr/http/redirection/metier/i...,10008,opérateur / opératrice prépresse,opérateur/trice prépresse,E1306,Prépresse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E1306


In [44]:
# Copie du tableau au format csv
output_path = os.path.join(dir_path, 'Métiers_Onisep_ROME.csv')
df.to_csv(output_path)

In [30]:
# Code Onisep des métiers avec des intitulés différents entre les fiches XML et le site internet
temp_list = []
for item in scraping_list:
    temp_list.append(item[0:6])

for item in parsing_list:
    if item[0:6] not in temp_list:
        print(item[0:6])

# Intitulé des métiers quand ils sont différents
c = 0
for row in df.itertuples():
    if row[1]!= row[5]:
        print(row[1], row[5])
        c+=1
print(c)

100703
100704
